In [7]:
import deepnets.optimization.save_load as save_load
import netket as nk
import netket_checkpoint as nkc
import json
import jax
import numpy as np

In [8]:
save_base = "/Users/Rajah.Nutakki/Desktop/work_dummy/test/" #where the output of a simulation with post_iterations is, see optimization

In [9]:
#Start of expectation value script
#Load in from checkpoint
min_index,system,network = save_load.load(save_base+"post.json","ConvNext")

In [10]:
min_index

8

In [11]:
#Load in the vstate
sampler = nk.sampler.MetropolisExchange(system.hilbert_space,graph=system.graph)
vstate = nk.vqs.MCState(sampler,model=network)
checkpoint = nkc.checkpoint.CheckpointManager(save_base + "post/checkpoint")
vstate = checkpoint.restore_state(vstate,step = min_index)

In [13]:
#Compute the expectation values
n_samples_per_chain = 1000
vstate.n_chains= 32
vstate.n_discard_per_chain=n_samples_per_chain
vstate.n_samples=n_samples_per_chain*vstate.n_chains
vstate.chunk_size = 500
operators = {
    "energy": system.hamiltonian,
}
results_dict={}
for name,operator in operators.items():
    result = vstate.expect(operator.to_jax_operator())
    result_dict = result.__dict__ #convert all of the attributes and their values to a dictionary
    #Convert to types compatible with json
    for key,value in result_dict.items():
        if isinstance(value,jax.Array):
            result_dict[key] = float(np.real(complex(value))) #cannot go directly from jax.Array with complex dtype to float, so take real part
    results_dict[name] = result_dict

#Save all results
with open(save_base+"expectation_values.json","w+") as f:
    json.dump(results_dict,f)

/var/folders/q5/3th0qwt539db5yhyjt0j0c1w0000gp/T/ipykernel_49698/4072019128.py:6: UserWarning: For performance reasons, we suggest to use a power-of-two chunk size.
  vstate.chunk_size = 500
